<a href="https://colab.research.google.com/github/inokchoi/Speech/blob/main/las.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

리스너튼 인코더, 스펠러는 디코더이다.
피라미드 아키텍쳐로 바이 디렉션 
임베딩 에이치
스펠러에 에이치 집어넣으면
케릭터로 나온다.

음성 루틴은 거의 같기 때문에 모델만 바꾸면 된다.
컨텍스트 찾아가는 부분 (C_i, S_i)

라스 티쳐프로세싱
stpo criterien이 구현이 안되어있다.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
cd '/content/drive/My Drive/Colab Notebooks/20-1voice_interface/Proj3/'

/content/drive/My Drive/Colab Notebooks/20-1voice_interface/Proj3


In [ ]:
!pip install editdistance
!pip install logger
!pip install tensorboard_logger
!pip install easydict

  Created wheel for logger: filename=logger-1.4-cp36-none-any.whl size=1789 sha256=22c413c0f3050f489912b8c92537ce80e08b6fa866abe84f5e119d032f890cba
  Stored in directory: /root/.cache/pip/wheels/91/d4/96/08341e2ac92c1ed4b760e4848e1acda3795f0257a83b94b42e
Successfully built logger


In [ ]:
import torch
import torch.nn.functional as F
import pdb
import numpy as np
import sys
import preprocess
import pdb
import os
import editdistance
import time
import easydict 
from torch.utils.data import DataLoader
from dataset import *
from model import *
from torch.optim import Adam
from tqdm import tqdm
from logger import Logger

# Store log information in the designated directory.
logger = Logger('./las_log/{}'.format(time.ctime()).replace(' ','_').replace(':','_'))

## Decode and LER calculation function

In [ ]:
def decode(pred, label, target_len, phn_list):
    ler = 0
    for i in range(pred.size(0)):
        # Find the maximum likelihood output.
        pred_class = torch.argmax(pred[i,:,:], dim=-1)
        
        # Collapse the repeated outputs. (Note: simplified implementation)
        collapse = [pred_class[k] for k in range(len(pred_class)) if (k==0) or pred_class[k] != pred_class[k-1]]
        
        # Compute the edit distance between the reference and estimated ones.
        collapse = torch.tensor([value for value in collapse if value != 0])        
        ed = editdistance.eval(collapse, label[i][:target_len[i]])/target_len[i]        
        ler += ed
        
    ler /= pred.size(0)
    
    return ler

## Inference function

In [ ]:
def inference(epoch, model, data_loader, criterion, optimizer, device, 
              phn_list, train=True):
    if train:
        model.train()
        flag = 'Train'
    else:
        model.eval()
        flag = 'Eval'

    total_loss = 0
    total_ler = 0
    progress_bar = tqdm(enumerate(data_loader))
    
    for b_idx, (data, label, input_lengs, target_lengs) in progress_bar:
        # Move data and label to the device (e.g. GPU).
        data = data.type(torch.FloatTensor).to(device)
        label = label.type(torch.LongTensor).to(device)
        
        # Predict the output given the input data.
        if train:
            pred, att_score, out_label = model(data, label, 1.0, train=train)
        else:
            with torch.no_grad():
                pred, att_score, out_label = model(data, label, 1.0, train=train)
                
        # Compute the loss by comparing the predicted output to the reference labels.
        loss = criterion(torch.log(pred).transpose(1,2), out_label[:,:pred.size(1)])
        total_loss += loss

        # Decode and calculate LER.
        ler = decode(pred, out_label, target_lengs, phn_list)
        total_ler += ler
        
        # Perform backpropagation if it is the training mode.
        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
          
        # Display the running progress.
        progress_bar.set_description(
            '{} Epoch:{}[{}/{}] CE_loss:{:.3f} LER:{:.3f} '.format( \
            flag, epoch, b_idx, len(data_loader), total_loss/(b_idx+1), 
            total_ler/(b_idx+1))
        )
        
    return total_loss, total_ler

## Main function

In [ ]:
def main():
    # Set the environmental parameters.
    args = easydict.EasyDict({
      'train_list':'./timit_train_list.txt',
      'test_list':'./timit_test_list.txt',
      'data_path':'./',
      'batch_size':4,
      'input_type':'mfcc',
      'input_dim':13,
      'delta_order':3,
      'resume':False,
      'resume_model':'las_10.pth',
      'lr':0.001,
      'max_epoch':100,
      'log_path':'./logs',
      'save_dir':'./models',
      
    })
  
    # Set the device for running the LAS model.
    device  = torch.device('cuda:0')

    hidden_dim = 256
    l_layer = 3
    s_layer = 1
    num_class = 40 + 1 + 1 +1      # 40 phoneme + <sos, eos, pad>  #pad 
    
    # Define a network architecture. ('model.py')
    model = LAS(args.input_dim*args.delta_order, hidden_dim, l_layer,
                hidden_dim+num_class, hidden_dim, num_class, s_layer)
    model = model.to(device)

    # Define an optimizer.
    optimizer = Adam(model.parameters(), lr=args.lr)

    # Define a training criterion.
    las_criterion = torch.nn.NLLLoss(ignore_index=0) # ignore <pad> labels  #C개 클래스를 클래시피케이션할 때 사용되는 것이다. negative log likelihood,

    # Load the pre-trained model if you resume the training from the model.
    os.makedirs(args.save_dir, exist_ok=True)
    if args.resume:
        if os.path.isfile(os.path.join(args.save_dir, args.resume_model)):
            checkpoint = torch.load(os.path.join(args.save_dir, args.resume_model),map_location=device)
            model.load_state_dict(checkpoint['encoder'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            start = checkpoint['epoch']
            print('----loading checkpoint----')
        else:
            print('can not find the resume model.')
    else:
        start=0

    # Set the phone list for decoding. ('preprocess.py')
    phn_list = preprocess.phn_list()

    # Define training and test datasets. ('dataset.py')
    train_dataset = ASR_Dataset(args.train_list, args.data_path, d_type=args.input_type, coeff=args.input_dim, delta=args.delta_order)
    train_loader = DataLoader(train_dataset,
                               batch_size = args.batch_size,
                               collate_fn = collate_fn,
                               shuffle=True,
                               num_workers=4,
                               pin_memory=True)

    test_dataset = ASR_Dataset(args.test_list, args.data_path, d_type=args.input_type, coeff=args.input_dim, delta=args.delta_order)
    test_loader = DataLoader(test_dataset,
                              batch_size=1,
                              collate_fn = collate_fn,
                              shuffle=False,
                              num_workers=4,
                              pin_memory=True)
    
    # Perform training/validation processing.
    for epoch in range(start, args.max_epoch+1):        
        train_loss, train_ler = inference(epoch, model, train_loader, las_criterion, 
                                          optimizer, device, phn_list, train=True)        
        valid_loss, valid_ler = inference(epoch, model, test_loader, las_criterion, 
                                          optimizer, device, phn_list, train=False)
        
        # Save the trained model at every 10 epochs.
        if epoch % 10 == 0:
            torch.save({'epoch':epoch, 'encoder':model.state_dict(),
                        'optimizer':optimizer.state_dict()},
                        '{}/las_{}.pth'.format(args.save_dir, epoch))

        # Log the loss and LER.
        logger.log_value('train_loss',train_loss)
        logger.log_value('train_ler',train_ler)
        logger.log_value('valid_loss',valid_loss)
        logger.log_value('valid_ler',valid_ler)
        logger.step()

In [ ]:
# run the main functoin
if __name__ == '__main__':
    main()

Train Epoch:0[227/228] CE_loss:3.226 LER:0.958 : : 228it [06:02,  1.59s/it]
Eval Epoch:0[294/295] CE_loss:3.193 LER:0.832 : : 295it [02:20,  2.09it/s]
Train Epoch:1[227/228] CE_loss:3.081 LER:0.620 : : 228it [03:00,  1.27it/s]
Eval Epoch:1[294/295] CE_loss:3.121 LER:0.594 : : 295it [01:04,  4.59it/s]
Train Epoch:2[227/228] CE_loss:2.958 LER:0.497 : : 228it [03:09,  1.20it/s]
Eval Epoch:2[175/295] CE_loss:3.064 LER:0.563 : : 176it [00:38,  5.36it/s]

KeyboardInterrupt: ignored

In [ ]:
librispeech 검색 : 데이터가 많이 있다.
이걸 가져와서 쓰면 큰 음성인식 시스템을 쓸 수 있다.

구글: las 많이 씀
아마존: las, ctc
대부분 최신 ai 스피커는 서버방식이므로 2개 중 하나 씀
일부 옛날 히든마크포모델+딥러닝 쓰는 경우도 있을 듯
ctc를 확장해서 쓰는 기관들이 더 많은 것 같다. (바이두, 페이스북)

언어별로 ctc, las중 잘 되는게  수 있다.


같은 모델이라도 어떤 하이퍼 파라미터를 쓰는지에달라질 따라 성능이 달라진다.
학습은 노하우가 필요하고,
학습이 좋으면 모델이 나빠도 성능을 높일 수 있다.